In [1]:
## ----------------------------------------------------------------------------
## Course: ECON 601
## Instructor: Michael Magill
## Problem: 1
## Author: Sam Boysel
## Date: 8/23/2017
## Desciption: Arrow-Debreu Equilibrium
## ----------------------------------------------------------------------------

## (1) Setup ------------------------------------------------------------------
# Pkg.add("JuMP")
# Pkg.add("Ipopt")

Pkg.update()

using JuMP, Clp, Ipopt

INFO: Updating METADATA...
INFO: Updating EconPDEs master...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [2]:
function init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
	m = Model(solver = IpoptSolver(print_level = 0)) 
	@variable(m, x_1_0 >= 0.0)
	@variable(m, x_1_1 >= 0.0)
	@variable(m, x_1_2 >= 0.0)
	@variable(m, x_1_3 >= 0.0)
	@variable(m, x_1_4 >= 0.0)
	# Agent 2
	@variable(m, x_2_0 >= 0.0)
	@variable(m, x_2_1 >= 0.0)
	@variable(m, x_2_2 >= 0.0)
	@variable(m, x_2_3 >= 0.0)
	@variable(m, x_2_4 >= 0.0)
	# Agent 3
	@variable(m, x_3_0 >= 0.0)
	@variable(m, x_3_1 >= 0.0)
	@variable(m, x_3_2 >= 0.0)
	@variable(m, x_3_3 >= 0.0)
	@variable(m, x_3_4 >= 0.0)
	# Prices
	@variable(m, π_0 == 1.0)
	@variable(m, π_1 >= 0.0)
	@variable(m, π_2 >= 0.0)
	@variable(m, π_3 >= 0.0)
	@variable(m, π_4 >= 0.0)
	# discount factor
	δ = 0.95
	## (1.2) Aggregate welfare --------------------------------------------
	@NLobjective(
		m,
		Max,
		# Agent 1
		(1 / (1 - α)) * (a[1] + x_1_0)^(1 - α) +
		δ * p[1, 1] * p[2, 1] * (1 / (1 - α)) * (a[1] + x_1_1)^(1 - α) +
		δ * p[1, 1] * (1 - p[2, 1]) * (1 / (1 - α)) * (a[1] + x_1_2)^(1 - α) +
		δ * (1 - p[1, 1]) * p[2, 1] * (1 / (1 - α)) * (a[1] + x_1_3)^(1 - α) +
		δ * (1 - p[1, 1]) * (1 - p[2, 1]) * (1 / (1 - α)) * (a[1] + x_1_4)^(1 - α) +
		# Agent 2
		(1 / (1 - α)) * (a[2] + x_2_0)^(1 - α) +
		δ * p[1, 2] * p[2, 2] * (1 / (1 - α)) * (a[2] + x_2_1)^(1 - α) +
		δ * p[1, 2] * (1 - p[2, 2]) * (1 / (1 - α)) * (a[2] + x_2_2)^(1 - α) +
		δ * (1 - p[1, 2]) * p[2, 2] * (1 / (1 - α)) * (a[2] + x_2_3)^(1 - α) +
		δ * (1 - p[1, 2]) * (1 - p[2, 2]) * (1 / (1 - α)) * (a[2] + x_2_4)^(1 - α) +
		# Agent 3
		(1 / (1 - α)) * (a[3] + x_3_0)^(1 - α) +
		δ * p[1, 3] * p[2, 3] * (1 / (1 - α)) * (a[3] + x_3_1)^(1 - α) +
		δ * p[1, 3] * (1 - p[2, 3]) * (1 / (1 - α)) * (a[3] + x_3_2)^(1 - α) +
		δ * (1 - p[1, 3]) * p[2, 3] * (1 / (1 - α)) * (a[3] + x_3_3)^(1 - α) +
		δ * (1 - p[1, 3]) * (1 - p[2, 3]) * (1 / (1 - α)) * (a[3] + x_3_4)^(1 - α)
	)
	## (1.3) Contraints
	## (1.3.1) State-contingent market clearing conditions ----------------
	# s = 0 
	@constraint(
		m,
		x_1_0 + x_2_0 + x_3_0 <= ω_1[1] + ω_2[1] + ω_3[1]
	)
	# s = 1 
	@constraint(
		m,
		x_1_1 + x_2_1 + x_3_1 <= ω_1[2] + ω_2[2] + ω_3[2]
	)
	# s = 2 
	@constraint(
		m,
		x_1_2 + x_2_2 + x_3_2 <= ω_1[3] + ω_2[3] + ω_3[3]
	)
	# s = 3 
	@constraint(
		m,
		x_1_3 + x_2_3 + x_3_3 <= ω_1[4] + ω_2[4] + ω_3[4]
	)
	# s = 4
	@constraint(
		m,
		x_1_4 + x_2_4 + x_3_4 <= ω_1[5] + ω_2[5] + ω_3[5]
	)
	## (1.3.2) Agent 1's state-contingent budget constraints --------------
	# s = 1
	@constraint(
		m,
		x_1_0 * π_0 + x_1_1 * π_1 <= ω_1[1] * π_0 + ω_1[2] * π_1
	)
	# s = 2
	@constraint(
		m,
		x_1_0 * π_0 + x_1_2 * π_2 <= ω_1[1] * π_0 + ω_1[3] * π_2
	)
	# s = 3
	@constraint(
		m,
		x_1_0 * π_0 + x_1_3 * π_3 <= ω_1[1] * π_0 + ω_1[4] * π_3
	)
	# s = 4
	@constraint(
		m,
		x_1_0 * π_0 + x_1_4 * π_4 <= ω_1[1] * π_0 + ω_1[5] * π_4
	)
	## (1.3.3) Agent 2's state-contingent budget constraints --------------
	# s = 1
	@constraint(
		m,
		x_2_0 * π_0 + x_2_1 * π_1 <= ω_2[1] * π_0 + ω_2[2] * π_1
	)
	# s = 2
	@constraint(
		m,
		x_2_0 * π_0 + x_2_2 * π_2 <= ω_2[1] * π_0 + ω_2[3] * π_2
	)
	# s = 3
	@constraint(
		m,
		x_2_0 * π_0 + x_2_3 * π_3 <= ω_2[1] * π_0 + ω_2[4] * π_3
	)
	# s = 4
	@constraint(
		m,
		x_2_0 * π_0 + x_2_4 * π_4 <= ω_2[1] * π_0 + ω_2[5] * π_4
	)
	## (1.3.4) Agent 3's state-contingent budget constraints --------------
	# s = 1
	@constraint(
		m,
		x_3_0 * π_0 + x_3_1 * π_1 <= ω_3[1] * π_0 + ω_3[2] * π_1
	)
	# s = 2
	@constraint(
		m,
		x_3_0 * π_0 + x_3_2 * π_2 <= ω_3[1] * π_0 + ω_3[3] * π_2
	)
	# s = 3
	@constraint(
		m,
		x_3_0 * π_0 + x_3_3 * π_3 <= ω_3[1] * π_0 + ω_3[4] * π_3
	)
	# s = 4
	@constraint(
		m,
		x_3_0 * π_0 + x_3_4 * π_4 <= ω_3[1] * π_0 + ω_3[5] * π_4
	)
	# non-zero prices
	slack = 1e-16
	@constraint(m, π_1 >= slack)
	@constraint(m, π_2 >= slack)
	@constraint(m, π_3 >= slack)
	@constraint(m, π_4 >= slack)
	X = [[x_1_0, x_1_1, x_1_2, x_1_3, x_1_4],
         [x_2_0, x_2_1, x_2_2, x_2_3, x_2_4],
         [x_3_0, x_3_1, x_3_2, x_3_3, x_3_4]]
    P = [π_0, π_1, π_2, π_3, π_4]
    m, X, P
end

# convenience function to print solutions
function print_solution(x, p)
    println("A-D Equilibrium:")
    println(" π = {π_0 = 1, ..., π_4}:")
    println(" - $(map(getvalue, p))")
    println(" x = {x_0, ..., x_4}")
    println(" - i = 1: $(map(getvalue, x[1]))")
    println(" - i = 2: $(map(getvalue, x[2]))")
    println(" - i = 3: $(map(getvalue, x[3]))")
end

print_solution (generic function with 1 method)

## (1)

First, assume $\epsilon_{i} \sim \text{iid}(\theta_{i}) > 0, \, \forall \, \theta_{i}$ in the parameter space and $i \in 1, 2$.  Second, denote the income stream to agent $i$ at time $1$ as $\nu^{i}_{s}$ where $s \in \{\text{success}, \text{failure}\}$ and
		\begin{align*}
			\nu^{i}_{\text{failure}} &= b_{i} - \epsilon_i \\
			\nu^{i}_{\text{success}} &= b_{i} + \epsilon_i
		\end{align*}
and hence $\nu^{i}_{\text{failure}} < \nu^{i}_{\text{success}}$ for $i \in 1, 2$.  Finally, note that the state space at $t = 1$ for agents 1 and 2 can be given by
		\begin{align*}
		S &= \{(\nu^{1}_{s}, \nu^{2}_{s})\}_{s \in \{\text{success},
		\text{failure}\}} \\
		&= \{(\nu^{1}_{\text{success}}, \nu^{2}_{\text{high}}),
		     (\nu^{1}_{\text{success}}, \nu^{2}_{\text{failure}}),
	     	 (\nu^{1}_{\text{failure}}, \nu^{2}_{\text{success}}),
	     	     (\nu^{1}_{\text{failure}}, \nu^{2}_{\text{failure}})\} \\
		     &= \{\nu^{1}_{\text{success}}, \nu^{1}_{\text{failure}}\} \times 
	     		\{\nu^{2}_{\text{success}}, \nu^{2}_{\text{failure}}\}	\\
		&= S_1 \times S_2
		\end{align*}

## (2)

Recognizing that

- $pr(k, i)$ represents $i$'s the subjective likelihood that firm $k$ is "successful" (e.g. $pr(\nu^{i}_{\text{success}})$)
- the probability distribution over the 4 states in $t = 1$ are formed by a joint distribution of two Bernoulli random variables (e.g. $pr(1, i): S_{1} \to [0, 1]$, $pr(2, i): S_{2} \to [0, 1]$, and $\rho^{i}: S \to [0, 1]$)

for each agent $i = 1, 2, 3$:

$$
\begin{align*}
\rho_{1}^{i} &= pr((\nu^{1}_{\text{high}}, \nu^{2}_{\text{high}})) = pr(1, i) \times pr(2, i) \\
\rho_{2}^{i} &= pr((\nu^{1}_{\text{high}}, \nu^{2}_{\text{low}})) = pr(1, i) \times (1 - pr(2, i)) \\
\rho_{3}^{i} &= pr((\nu^{1}_{\text{low}}, \nu^{2}_{\text{high}})) = (1 - pr(1, i)) \times pr(2, i) \\
\rho_{4}^{i} &= pr((\nu^{1}_{\text{low}}, \nu^{2}_{\text{low}})) = (1 - pr(1, i)) \times (1 - pr(2, i))
\end{align*}
$$


## (3) No risk, identical beliefs

### (a) Find the Arrow-Debreu Equilibrium (ADE)

In [3]:
## (3.1) Parameters
α = 2.0
a = [0.0, 0.0, 0.0]
# probabilities
# - p[k, i] = agent i's belief that firm k is successful
p = [0.5 0.5 0.5;
     0.5 0.5 0.5]
# non-stochastic wealth
b = [500.0, 500.0, 1000.0]
# stochastic wealth
ϵ = [0.0, 0.0]
# endowment vectors
ω_1 = [0.0, b[1] + ϵ[1], b[1] + ϵ[1], b[1] - ϵ[1], b[1] - ϵ[1]]
ω_2 = [0.0, b[2] + ϵ[2], b[2] - ϵ[2], b[2] + ϵ[2], b[2] - ϵ[2]]
ω_3 = [b[3], 0.0, 0.0, 0.0, 0.0]

# Initiate model and solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 2.00001, 2.00001, 2.00001, 2.00001]
 x = {x_0, ..., x_4}
 - i = 1: [333.334, 333.333, 333.333, 333.333, 333.333]
 - i = 2: [333.334, 333.333, 333.333, 333.333, 333.333]
 - i = 3: [333.332, 333.33, 333.33, 333.33, 333.33]


### (3) (b) Explain why ADE prices are constant at $t = 1$ and how they relate to $\delta$ and $\rho$.

### (3) (c) Net trades

Explain net trades $\bar{\tau}^{i} = \bar{x}^{i} - \bar{\omega}^{i}$.  Could the outcome be achieved by borrowing and elnding alone?

### (3) (d) Calculate implied interest rate at the ADE

## (4) Some risk, identical beliefs

### (a) Find ADE

Find ADE and show that optimal prices in are related to aggregate output in $t = 1$.

In [4]:
## (4.1) Parameters
# stochastic wealth
ϵ = [100.0, 200.0]
# endowment vectors
ω_1 = [0.0, b[1] + ϵ[1], b[1] + ϵ[1], b[1] - ϵ[1], b[1] - ϵ[1]]
ω_2 = [0.0, b[2] + ϵ[2], b[2] - ϵ[2], b[2] + ϵ[2], b[2] - ϵ[2]]
ω_3 = [b[3], 0.0, 0.0, 0.0, 0.0]

# Initiate model and solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 1.57219, 2.72719, 2.1694, 2.9521]
 x = {x_0, ..., x_4}
 - i = 1: [346.662, 379.502, 472.882, 240.203, 282.57]
 - i = 2: [320.18, 496.344, 182.597, 552.405, 191.541]
 - i = 3: [333.157, 424.146, 244.516, 307.385, 225.887]


### (4) (b)

Show that the consumption stream for agent $i$ at $t = 1$, denoted $\bar{x}_{1}^{i} \in \mathbb{R}^4$ covaries with aggregate output at $t = 1$, denoted $\omega_{1} \in \mathbb{R}^4 = \sum_{i = 1}^{3} \omega_{1}^{i}$.  Furthermore, show that $\exists (c_i, d_i) \, \forall i = 1, 2, 3$ such that
$$
\begin{align}
\sum_{i = 1}^{3}c_i &= 0 \\
\sum_{i = 1}^{3}d_i &= 1 \\
\bar{x}^{i}_{1} &= c_{i}\mathbb{1} + d_i \omega_1
\end{align}
$$

### (4) (c) Net trades

Explain net trades $\bar{\tau}^{i} = \bar{x}^{i} - \bar{\omega}^{i}$.  Could the outcome be achieved by borrowing and lending alone?

## (5) Some risk, variation in investor beliefs

### (5) (a) Find ADE

In [5]:
## (5.1) Parameters
# probabilities
# - p[k, i] = agent i's belief that firm k is successful
p = [0.5 0.5 2/3;
     0.5 0.5 2/3]

# Initiate model solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 1.30652, 2.86567, 2.22623, 3.94754]
 x = {x_0, ..., x_4}
 - i = 1: [339.357, 340.257, 481.574, 247.564, 314.032]
 - i = 2: [327.347, 449.448, 185.769, 552.954, 217.075]
 - i = 3: [333.296, 510.289, 232.651, 299.475, 168.89]


### (5) (b)

How has equilibrium changed relative to (4)?

## (6) Some risk, variation in preferences

### (6) (a) Find ADE

Show that $\exists (c_i, d_i) \, \forall i = 1, 2, 3$ such that
$$
\begin{align}
\sum_{i = 1}^{3}c_i &= 0 \\
\sum_{i = 1}^{3}d_i &= 1 \\
x^{i}_{1} &= c_{i}\mathbb{1} + d_i \omega_1
\end{align}
$$

In [6]:
## (6.1) Parameters
a = [0.0, 0.0, 500.0]
# probabilities
# - p[k, i] = agent i's belief that firm k is successful
p = [0.5 0.5 0.5;
     0.5 0.5 0.5]
# non-stochastic wealth
b = [500.0, 500.0, 1000.0]
# stochastic wealth
ϵ = [100.0, 200.0]
# endowment vectors
ω_1 = [0.0, b[1] + ϵ[1], b[1] + ϵ[1], b[1] - ϵ[1], b[1] - ϵ[1]]
ω_2 = [0.0, b[2] + ϵ[2], b[2] - ϵ[2], b[2] + ϵ[2], b[2] - ϵ[2]]
ω_3 = [b[3], 0.0, 0.0, 0.0, 0.0]

# Initiate model and solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 35345.1, 43770.2, 39139.6, 48227.1]
 x = {x_0, ..., x_4}
 - i = 1: [499.884, 583.895, 582.061, 392.467, 391.037]
 - i = 2: [499.881, 678.067, 295.52, 677.115, 295.138]
 - i = 3: [0.234262, 0.0153233, 0.0127065, 0.0140188, 0.0115977]


### (6) (b)

To what numbers do the coefficients $d_i$ converge when $a^{3} \to \infty$?  What is happening to risk sharing?

## (7) Differences in beliefs

### (7) (a) Find ADE under identical beliefs

Explain the resulting ADE

In [7]:
a = [0.0, 0.0, 0.0]
# probabilities
# - p[http://localhost:8888/notebooks/p1.ipynb#k, i] = agent i's belief that firm k is successful
p = [0.5 0.5 0.5;
     0.5 0.5 0.5]
# non-stochastic wealth
b = [500.0, 500.0, 500.0]
# stochastic wealth
ϵ = [0.0, 0.0]
# endowment vectors
ω_1 = [0.0, b[1] + ϵ[1], b[1] + ϵ[1], b[1] - ϵ[1], b[1] - ϵ[1]]
ω_2 = [0.0, b[2] + ϵ[2], b[2] - ϵ[2], b[2] + ϵ[2], b[2] - ϵ[2]]
ω_3 = [b[3], b[3], b[3], b[3], b[3]]

# Initiate model and solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 2.15695e5, 2.15695e5, 2.15695e5, 2.15695e5]
 x = {x_0, ..., x_4}
 - i = 1: [166.667, 499.995, 499.995, 499.995, 499.995]
 - i = 2: [166.667, 499.995, 499.995, 499.995, 499.995]
 - i = 3: [166.667, 499.997, 499.997, 499.997, 499.997]


### (7) (b) Find ADE under heterogenous beliefs

Explain why this ADE differs from (7)(a).

In [8]:
# parameters
a = [0.0, 0.0, 0.0]
# probabilities
# - p[k, i] = agent i's belief that firm k is successful
p = [2/3 1/3 3/4;
     2/3 1/2 1/4]
# non-stochastic wealth
b = [500.0, 500.0, 500.0]
# stochastic wealth
ϵ = [0.0, 0.0]
# endowment vectors
ω_1 = [0.0, b[1] + ϵ[1], b[1] + ϵ[1], b[1] - ϵ[1], b[1] - ϵ[1]]
ω_2 = [0.0, b[2] + ϵ[2], b[2] - ϵ[2], b[2] + ϵ[2], b[2] - ϵ[2]]
ω_3 = [b[3], b[3], b[3], b[3], b[3]]

# Initiate model and solve for equilibrium
m, X, P = init_model(α, a, p, b, ϵ, ω_1, ω_2, ω_3)
status = solve(m)
print_solution(X, P)

A-D Equilibrium:
 π = {π_0 = 1, ..., π_4}:
 - [1.0, 2.17464e5, 217154.0, 2.01101e5, 2.07378e5]
 x = {x_0, ..., x_4}
 - i = 1: [166.667, 499.997, 499.995, 499.996, 499.989]
 - i = 2: [166.667, 499.993, 499.993, 499.997, 499.997]
 - i = 3: [166.667, 499.996, 500.0, 499.983, 499.997]


## (8) Summary of qualitative insights